# 第6篇：合并和联接

在连接/合并类型操作的情况下，pandas提供了多种功能，可以轻松地将Series或DataFrame与各种用于索引和关系代数功能的集合逻辑组合在一起。用的比较多的有三个函数：
- `pandas.merge/pandas.DataFrame.merge`: 横向连接两个DataFrame,可以通过索引或者列
- `pandas.DataFrame.join`: 横向联接两个DataFrame，默认以公共index进行联接，左边的DataFrame可以指定列与右边DataFrame的index进行联接，而右边不能指定列。
- `pandas.concat`: 将多个DataFrame进行纵向合并/横向联接，axis为1时表示横向联接，且只能以公共的index进行联接，不能指定公共列。

## 第1部分：合并

导入相关库

In [334]:
import numpy as np
import pandas as pd

In [335]:
data1 = {
    "name": pd.Series(["Tom", "Bob", "Mary", "James", "Yafei"]),
    "age": pd.Series([18, 30, 25, 40, 22]),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"])
}
df1 = pd.DataFrame(data1)
df1

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


In [336]:
data2 = {
    "name": pd.Series(["Kobe", "Allen", "Machael"]),
    "age": pd.Series([41, 52, 43]),
    "city": pd.Series(["洛杉矶", "费城", "芝加哥"])
}
df2 = pd.DataFrame(data2)
df2

,name,age,city
0,Kobe,41,洛杉矶
1,Allen,52,费城
2,Machael,43,芝加哥


上面有两个DataFrame：df1和df2，我们发现其均有三列且列名相同，一个是5个人的信息，一个是3个人的信息，那么如何将两个DataFrame的数据进行纵向合并呢？

### concat
> concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True):

concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接
所谓外连接，就是取拼接方向的并集，而'inner'时取拼接方向（若使用默认的纵向拼接，则为列的交集）的交集。下面通过一些例子理解concat的用法和各个参数的意义。

In [337]:
pd.concat(objs=[df1, df2])

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城
0,Kobe,41,洛杉矶
1,Allen,52,费城
2,Machael,43,芝加哥


In [338]:
pd.concat(objs=[df1, df2], ignore_index=True)

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城
5,Kobe,41,洛杉矶
6,Allen,52,费城
7,Machael,43,芝加哥


追加新行

In [339]:
# list
pd.concat([df1, pd.DataFrame([["Tony", 40, "苏州"]], columns=df1.columns)], ignore_index=True)

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城
5,Tony,40,苏州


In [340]:
# dict
pd.concat([df1, pd.DataFrame({"name":["Tony"], "age":[40], "city": ["苏州"]})], ignore_index=True)

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城
5,Tony,40,苏州


### assign

In [341]:
df1

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


In [342]:
data2 = {
    "hobby": pd.Series(['羽毛球', '棒球', '舞蹈', '看球', '跑步']),
    "sex": pd.Series(['男', '男', '女', '男', '男'])
}
df2 = pd.DataFrame(data2)
df2

,hobby,sex
0,羽毛球,男
1,棒球,男
2,舞蹈,女
3,看球,男
4,跑步,男


横向合并

In [343]:
df1.assign(hobby=df2['hobby'], sex=df2['sex'])

,name,age,city,hobby,sex
0,Tom,18,北京,羽毛球,男
1,Bob,30,上海,棒球,男
2,Mary,25,广州,舞蹈,女
3,James,40,深圳,看球,男
4,Yafei,22,晋城,跑步,男


In [344]:
df1.assign(**{col: df2[col] for col in df2.columns})

,name,age,city,hobby,sex
0,Tom,18,北京,羽毛球,男
1,Bob,30,上海,棒球,男
2,Mary,25,广州,舞蹈,女
3,James,40,深圳,看球,男
4,Yafei,22,晋城,跑步,男


## 第2部分：横向联接

### merge
> merge(right, how='inner', on=None, left_on=None, right_on=None,left_index=False, right_index=False, sort=False,suffixes=('_x', '_y'), copy=True, indicator=False,validate=None)

- how: 连接方式，默认how为inner,即内连接。
- on为公共列，若公共列名相同，以on为公共列对两个df进行连接，若不同，可以指定left_on和right_on。on可以为一个公共列，也可以为多个公共列，多个用列表表示。
- left_index, right_index: 是否以索引对齐进行连接，默认为False。
- sort: 是否排序
- suffixes: 重复的列后缀命名方式，默认为['_x', '_y']
- indicatos: 是否显示改行索引的来源，默认不显示
- validate: 如果为True,检查两边的索引属于哪种类型，有one_to_one（两边都唯一）/one_to_many（左唯一，右重复）/many_to_one（左重复，右唯一）/many_to_many(左右都重复)三种。

In [345]:
df1

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


In [346]:
data2 = {
    "name": pd.Series(["Bob", "Mary", "James", "Kobe", "Allen"]),
    "age": pd.Series([30, 25, 40, 41, 41]),
    "hobby": pd.Series(['羽毛球', '棒球', '舞蹈', '看球', '跑步']),
    "sex": pd.Series(['男', '男', '女', '男', '男'])
}
df2 = pd.DataFrame(data2)
df2

,name,age,hobby,sex
0,Bob,30,羽毛球,男
1,Mary,25,棒球,男
2,James,40,舞蹈,女
3,Kobe,41,看球,男
4,Allen,41,跑步,男


on：公共列，suffixes: 重复列后缀

In [347]:
df3 = df1.merge(df2, on='name', how='inner')
df3

,name,age_x,city,age_y,hobby,sex
0,Bob,30,上海,30,羽毛球,男
1,Mary,25,广州,25,棒球,男
2,James,40,深圳,40,舞蹈,女


In [348]:
df3 = df1.merge(df2, on='name', how='inner', suffixes=['-df1', '-df2'])
df3

,name,age-df1,city,age-df2,hobby,sex
0,Bob,30,上海,30,羽毛球,男
1,Mary,25,广州,25,棒球,男
2,James,40,深圳,40,舞蹈,女


validate检验的是到底哪一边出现了重复索引，如果是“one_to_one”则两侧索引都是唯一，如果"one_to_many"则左侧唯一

In [349]:
df3 = df1.merge(df2, on='age', how='inner')
df3

,name_x,age,city,name_y,hobby,sex
0,Bob,30,上海,Bob,羽毛球,男
1,Mary,25,广州,Mary,棒球,男
2,James,40,深圳,James,舞蹈,女


In [350]:
# df3 = df1.merge(df2, on='age', how='right', validate='one_to_one')
# ---------------------------------------------------------------------------
# MergeError                                Traceback (most recent call last)
# <ipython-input-41-0735a25e52d7> in <module>
# ----> 1 df3 = df1.merge(df2, on='age', how='inner', validate='one_to_one')
#       2 df3
# ...
# MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [351]:
df3 = df1.merge(df2, on='age', how='right', validate='one_to_many')
df3

,name_x,age,city,name_y,hobby,sex
0,Bob,30,上海,Bob,羽毛球,男
1,Mary,25,广州,Mary,棒球,男
2,James,40,深圳,James,舞蹈,女
3,NaN,41,NaN,Kobe,看球,男
4,NaN,41,NaN,Allen,跑步,男


on也可以是多个公共列

In [352]:
df3 = df1.merge(df2, on=['name', 'age'], how='inner')
df3

,name,age,city,hobby,sex
0,Bob,30,上海,羽毛球,男
1,Mary,25,广州,棒球,男
2,James,40,深圳,舞蹈,女


有时候，两个 DataFrame 中需要关联的键的名称不一样，可以通过 left_on 和 right_on 来分别设置。

In [353]:
df11 = df1.rename(columns={"name": "name1", "age": 'age1'})
df21 = df2.rename(columns={"name": "name2", 'age': 'age2'})
df11

,name1,age1,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


In [354]:
df21

,name2,age2,hobby,sex
0,Bob,30,羽毛球,男
1,Mary,25,棒球,男
2,James,40,舞蹈,女
3,Kobe,41,看球,男
4,Allen,41,跑步,男


In [355]:
df3 = df11.merge(df21, left_on=['name1', 'age1'], right_on=['name2', 'age2'])
df3

,name1,age1,city,name2,age2,hobby,sex
0,Bob,30,上海,Bob,30,羽毛球,男
1,Mary,25,广州,Mary,25,棒球,男
2,James,40,深圳,James,40,舞蹈,女


关联后发现数据变少了，只有 3 行数据，这是因为默认关联的方式是 inner，如果不想丢失任何数据，可以设置参数 howw"outer",如果我们想保留左边所有的数据，可以设置参数 how="left"；反之，如果想保留右边的所有数据，可以设置参数 how="right"

左连接

In [356]:
df3 = df1.merge(df2, on=['name', 'age'], how='left')
df3

,name,age,city,hobby,sex
0,Tom,18,北京,NaN,NaN
1,Bob,30,上海,羽毛球,男
2,Mary,25,广州,棒球,男
3,James,40,深圳,舞蹈,女
4,Yafei,22,晋城,NaN,NaN


右连接

In [357]:
df3 = df1.merge(df2, on=['name', 'age'], how='right')
df3

,name,age,city,hobby,sex
0,Bob,30,上海,羽毛球,男
1,Mary,25,广州,棒球,男
2,James,40,深圳,舞蹈,女
3,Kobe,41,NaN,看球,男
4,Allen,41,NaN,跑步,男


外连接

In [358]:
df3 = df1.merge(df2, on=['name', 'age'], how='outer')
df3

,name,age,city,hobby,sex
0,Tom,18,北京,NaN,NaN
1,Bob,30,上海,羽毛球,男
2,Mary,25,广州,棒球,男
3,James,40,深圳,舞蹈,女
4,Yafei,22,晋城,NaN,NaN
5,Kobe,41,NaN,看球,男
6,Allen,41,NaN,跑步,男


indicator参数指示了，合并后该行索引的来源

In [359]:
df3 = df1.merge(df2, on=['name', 'age'], how='outer', indicator=True)
df3

,name,age,city,hobby,sex,_merge
0,Tom,18,北京,NaN,NaN,left_only
1,Bob,30,上海,羽毛球,男,both
2,Mary,25,广州,棒球,男,both
3,James,40,深圳,舞蹈,女,both
4,Yafei,22,晋城,NaN,NaN,left_only
5,Kobe,41,NaN,看球,男,right_only
6,Allen,41,NaN,跑步,男,right_only


### join
> join(other, on=None, how='left', lsuffix='', rsuffix='',sort=False)

除了 merge 这种方式外，还可以通过 join 这种方式实现关联。join函数作用是将多个pandas对象横向拼接，遇到重复的索引项时会使用笛卡尔积，默认左连接，可选inner、outer、right连接相比 merge，join 这种方式有以下几个不同：
- 默认参数on=None，表示关联时使用左边和右边的索引作为键，设置参数on可以指定的是关联时左边的所用到的键名
- 左边和右边字段名称重复时，通过设置参数 lsuffix 和 rsuffix 来解决。

In [360]:
df1.join(df2.set_index('name'), on='name', lsuffix='-df1', rsuffix='-df2')

,name,age-df1,city,age-df2,hobby,sex
0,Tom,18,北京,NaN,NaN,NaN
1,Bob,30,上海,30.0,羽毛球,男
2,Mary,25,广州,25.0,棒球,男
3,James,40,深圳,40.0,舞蹈,女
4,Yafei,22,晋城,NaN,NaN,NaN


In [361]:
df1.join(df2.set_index(['name', 'age']), on=['name', 'age'], lsuffix='-df1', rsuffix='-df2')

,name,age,city,hobby,sex
0,Tom,18,北京,NaN,NaN
1,Bob,30,上海,羽毛球,男
2,Mary,25,广州,棒球,男
3,James,40,深圳,舞蹈,女
4,Yafei,22,晋城,NaN,NaN


注：连接的df2必须指定索引进行连接，df1可以是索引，也可以是指定列，所为指定列时必须指定on

### concat

> concat(objs, axis=1, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True):

In [362]:
pd.concat(objs=[df1.set_index('name'), df2.set_index('name')], join='inner', axis=1)

,age,city,age,hobby,sex
name,,,,,
Bob,30,上海,30,羽毛球,男
Mary,25,广州,25,棒球,男
James,40,深圳,40,舞蹈,女


注：concat方法对两个df进行横向连接时，axis必须设置为1,只能以两个df共同的index进行连接，即不能指定公共列，连接方式可以通过指定join参数，默认为outer，即外连接。

## 第3部分：融汇贯通

将上述学到的合并联接方法整理成通用函数，下面是我整理的，可以参考。

```python
from pandas import read_csv, read_excel, merge, concat, DataFrame


def read_file(file_path, on):
    if file_path.endswith('.csv'):
        return read_csv(file_path)
    if file_path.endswith('.xls') or file_path.endswith('xlsx'):
        return read_excel(file_path)


def df_to_file(df: DataFrame, file_path: str, index: bool = True, encoding: str = 'utf_8_sig'):
    if file_path.endswith('.csv'):
        df.to_csv(file_path, index=index, encoding=encoding)
    if file_path.endswith('.xls') or file_path.endswith('xlsx'):
        df.to_excel(file_path, index=index)


def merge_two_data(file1: str, file2: str, on: str = None, left_on: str = None, right_on: str = None,
                   how: str = 'inner', to_file: str = None):
    """
    横向合并两个文件
    @param file1:
    @param file2:
    @param on:
    @param left_on:
    @param right_on:
    @param how:
    @param to_file:
    @return:
    """
    df1 = read_file(file1)
    df2 = read_file(file2)
    merge_df = merge(df1, df2, on=on, how=how, left_on=left_on, right_on=right_on)
    if to_file:
        if to_file.endswith('.csv'):
            merge_df.to_csv(to_file, encoding='utf_8_sig', index=False)
        elif to_file.endswith('xls') or to_file.endswith('xlsx'):
            merge_df.to_excel(to_file, index=False)
    else:
        return merge_df


def append_two_file(file1: str, file2: str, ignore_index: bool = True, to_file: str = None):
    """
    纵向合并两个文件
    @param file1:
    @param file2:
    @param to_file:
    @return:
    """
    df1 = read_file(file1)
    df2 = read_file(file2)
    df3 = df1.append(df2, ignore_index=ignore_index)
    if to_file:
        df_to_file(df3, to_file, index=False)
    else:
        return df3


def join_two_file(file1: str, file2: str, on=None, left_on=None, right_on=None, how: str = 'left', to_file: str = None):
    """
    横向联接两个表格文件
    @param file1:
    @param file2:
    @param on:
    @param left_on:
    @param right_on:
    @param how:
    @param to_file:
    @return: None or DataFrame
    """
    if on:
        df1 = read_file(file1).set_index(keys=on)
        df2 = read_file(file2).set_index(keys=on)
    elif left_on or right_on:
        df1 = read_file(file1).set_index(keys=on) if left_on else read_file(file1)
        df2 = read_file(file2).set_index(keys=on) if right_on else read_file(file2)
    else:
        df1 = read_file(file1)
        df2 = read_file(file2)
    df3 = df1.join(df2, how=how)
    if to_file:
        df_to_file(df3, to_file, index=False)
    else:
        return df3


def concat_mul_file(axis: int = 0, on=None, to_file: str = None, encoding: str = 'utf_8_sig', *files):
    """
    多个表格文件合并
    @param axis: 0/index 1/column 若axis=1, 默认基于索引将多个文件合并
    @param on: 当axis=1时，指定索引列/索引列
    @param to_file: 导出文件路径
    @param encoding: 导出文件编码
    @param files: 合并文件路径
    @return:
    """
    if len(files) > 1:
        if axis == 1 and on:
            objs = [read_file(file).set_index(keys=on) for file in files]
        else:
            objs = [read_file(file) for file in files]
        merge_data = concat(objs=objs, axis=axis)
        if to_file:
            df_to_file(merge_data, to_file, index=False, encoding=encoding)
        else:
            return merge_data
    else:
        raise Exception('合并的文件个数小于2，不能进行合并，请输入大于等于两个文件路径')
 ```

## 简单小结
数据合并和联接是数据处理中常用的操作技巧，pandas为相关操作提供了非常方便的方法，通过本节的内容，可以学会利用这些方法并在实际工作中高效的提高你的效率，这也是我们学习编程的意义之一，就是学以致用。